In [30]:
"""
Новая версия.
"""

'\nНовая версия.\n'

In [31]:
import tkinter
import sys
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
import datetime
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn'


In [45]:
name_file_abiturs = 'data/abitur.xlsx'
name_file_person = 'data/person.xlsx'
path_to_end_folder_report = 'data'

In [46]:
#создаем датафрейм со специальностями
lst_code = ['23.01.09','43.01.06','23.02.06','43.02.06','15.01.05','15.01.35','15.01.33','23.01.10'
    ,'08.01.31','23.01.17','08.02.09','23.02.07','13.02.07','35.01.27']

lst_name_spec = ['Машинист локомотива','Проводник на железнодорожном транспорте','Техническая эксплуатация подвижного состава железных дорог',
                 'Сервис на транспорте (по видам транспорта)','Сварщик (ручной и частично механизированной сварки (наплавки)',
                 'Мастер слесарных работ','Токарь на станках с числовым программным управлением','Слесарь по обслуживанию и ремонту подвижного состава'
    ,'Электромонтажник электрических сетей и электрооборудования','Мастер по ремонту и обслуживанию автомобилей',
                 'Монтаж, наладка и эксплуатация электрооборудования промышленных и гражданских зданий',
                 'Техническое обслуживание и ремонт двигателей, систем и агрегатов автомобилей','Электроснабжение (по отраслям)',
                 'Мастер сельскохозяйственного производства']

lst_plan = [100,25,25,25,50,25,25,50
    ,25,25,25,25,25,25]
base_df = pd.DataFrame(columns=['Код','Наименование'])
base_df['Код'] = lst_code
base_df['Наименование'] = lst_name_spec
base_df['Направление подготовки'] = base_df['Код'] + ' ' + base_df['Наименование']
base_df['База'] = '9 кл.'
base_df['Количество мест'] = lst_plan

In [65]:
df_abitur = pd.read_excel(name_file_abiturs, skiprows=3, usecols=['Абитуриент', 'Доп. статус', 'Состояние'])
df_person = pd.read_excel(name_file_person, sheet_name='Абитуриенты', skiprows=8,
                          usecols=['ФИО', 'Нуждается в общежитии', 'Формирующее подр.',
                                   'Направление подготовки', 'Сдан оригинал'])

df_person = df_person[~df_person['Направление подготовки'].isnull()]  # убираем тех у кого нет заявлений

In [66]:
ts=df_person[df_person['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

In [67]:
ts.shape

(19, 5)

In [68]:
df_dupl = df_person.drop_duplicates(subset='ФИО')  # создаем датафрейм без дубликатов


dupl_cross_df = df_dupl.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')


In [69]:
ts2=dupl_cross_df[dupl_cross_df['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

In [70]:
ts2.shape

(19, 8)

In [71]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
dupl_cross_df['Нуждается в общежитии'] = dupl_cross_df['Нуждается в общежитии'].apply(
    lambda x: 0 if x == 'нет' else 1)
dupl_cross_df['Сдан оригинал'] = dupl_cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)
dupl_cross_df['Состояние'] = dupl_cross_df['Состояние'].apply(lambda x: 1 if x == 'Забрал документы' else 0)

# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
dupl_cross_df['Доп. статус'].fillna('', inplace=True)
dupl_cross_df['Сироты'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
dupl_cross_df['СВО'] = dupl_cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)
dupl_cross_df['Целевой договор'] = dupl_cross_df['Доп. статус'].apply(lambda x: 1 if 'Целевой договор' in x else 0)

dupl_cross_df['for_counting'] = 1

dupl_cross_df.drop(columns=['Доп. статус'], inplace=True)

In [72]:
ts3=dupl_cross_df[dupl_cross_df['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

In [73]:
ts3.shape

(19, 11)

In [74]:
dupl_svod_df = pd.DataFrame.pivot_table(dupl_cross_df,
                                        index=['Формирующее подр.', 'Направление подготовки'],
                                        values=['for_counting', 'Состояние', 'Сдан оригинал', 'Сироты', 'СВО','Целевой договор',
                                                'Нуждается в общежитии'],
                                        aggfunc='sum')

dupl_svod_df.columns = ['Заявлений', 'Нуждается в общежитии чел.', 'Дети СВО', 'Сдано оригиналов', 'Сирот чел.',
                        'Забрали заявления','Целевой договор']

dupl_svod_df['Итого заявлений'] = dupl_svod_df['Заявлений'] - dupl_svod_df['Забрали заявления']

dupl_svod_df['Итого заявлений'] = dupl_svod_df['Заявлений'] - dupl_svod_df['Забрали заявления']
# Меняем местами столбцы
single_out_df = dupl_svod_df.reindex(
    columns=['Заявлений', 'Забрали заявления', 'Итого заявлений', 'Сдано оригиналов',
             'Нуждается в общежитии чел.',
             'Сирот чел.', 'Дети СВО','Целевой договор'])


In [77]:
# Соединяем оба датафрейма

cross_df = df_person.merge(df_abitur, how='inner', left_on='ФИО', right_on='Абитуриент')

In [80]:
cross_df[cross_df['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

,ФИО,Нуждается в общежитии,Формирующее подр.,Направление подготовки,Сдан оригинал,Абитуриент,Доп. статус,Состояние
25,Арефьев Леонид Михайлович,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Арефьев Леонид Михайлович,NaN,Сданы ВИ
188,Горбунов Тимур Сергеевич,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Горбунов Тимур Сергеевич,NaN,Сданы ВИ
240,Дондоков Соёл Сергеевич,нет,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Дондоков Соёл Сергеевич,NaN,Сданы ВИ
276,Зимирева Евгения Викторовна,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Зимирева Евгения Викторовна,NaN,Сданы ВИ
280,Иванов Александр Владимирович,нет,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Иванов Александр Владимирович,Сирота; (c 21.12.2015),Сданы ВИ
333,Кисилева Елизавета Андреевна,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Кисилева Елизавета Андреевна,NaN,NaN
334,Кисилева Елизавета Андреевна,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Кисилева Елизавета Андреевна,Сирота; (c 27.01.2022),Сданы ВИ
370,Корнакова Дарья Николаевна,нет,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Корнакова Дарья Николаевна,NaN,Сданы ВИ
376,Корнилова Эльвира Анатольевна,нет,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Корнилова Эльвира Анатольевна,NaN,Сданы ВИ
395,Краснопеева Светлана Антоновна,да,Хоринский филиал,35.01.27 Мастер сельскохозяйственного производ...,да,Краснопеева Светлана Антоновна,NaN,Сданы ВИ


In [81]:
ts4=cross_df[cross_df['Направление подготовки'] == '35.01.27 Мастер сельскохозяйственного производства']

In [82]:
ts4.shape

(20, 8)

In [61]:


# Преобразовываем да-нет в 1 или 0 для подсчетов
cross_df['Нуждается в общежитии'] = cross_df['Нуждается в общежитии'].apply(lambda x: 0 if x == 'нет' else 1)
cross_df['Сдан оригинал'] = cross_df['Сдан оригинал'].apply(lambda x: 0 if x == 'нет' else 1)
cross_df['Состояние'] = cross_df['Состояние'].apply(lambda x: 1 if x == 'Забрал документы' else 0)

# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
cross_df['Доп. статус'].fillna('', inplace=True)
cross_df['Сироты'] = cross_df['Доп. статус'].apply(lambda x: 1 if 'Сирота;' in x else 0)
cross_df['СВО'] = cross_df['Доп. статус'].apply(
    lambda x: 1 if 'Дети военнослужащих, участвующих в спецоперации' in x else 0)

cross_df['for_counting'] = 1

cross_df.drop(columns=['Доп. статус'], inplace=True)

In [62]:
cross_df.columns

Index(['ФИО', 'Нуждается в общежитии', 'Формирующее подр.',
       'Направление подготовки', 'Сдан оригинал', 'Абитуриент', 'Состояние',
       'Сироты', 'СВО', 'for_counting'],
      dtype='object')

In [63]:
ts3 = cross_df[cross_df['Направление подготовки']=='35.01.27 Мастер сельскохозяйственного производства']

In [64]:
ts3.shape

(20, 10)

In [35]:


# Создаем сокращенный датафрейм чтобы добавить его в базовый
small_df = cross_df[['Направление подготовки','for_counting']]

# объединяем датафреймы
union_df = base_df.merge(small_df,how='outer',left_on='Направление подготовки',right_on='Направление подготовки')
union_df.fillna(0,inplace=True)

In [41]:
union_df

,Код,Наименование,Направление подготовки,База,Количество мест,for_counting
0,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100.0,1
1,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100.0,1
2,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100.0,1
3,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100.0,1
4,23.01.09,Машинист локомотива,23.01.09 Машинист локомотива,9 кл.,100.0,1
...,...,...,...,...,...,...
830,0,0,38.02.02 Страховое дело (по отраслям),0,0.0,1
831,0,0,38.02.02 Страховое дело (по отраслям),0,0.0,1
832,0,0,38.02.02 Страховое дело (по отраслям),0,0.0,1
833,0,0,38.02.02 Страховое дело (по отраслям),0,0.0,1


In [44]:
union_df[union_df['Наименование'] =='Мастер сельскохозяйственного производства']

,Код,Наименование,Направление подготовки,База,Количество мест,for_counting
806,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
807,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
808,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
809,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
810,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
811,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
812,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
813,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
814,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1
815,35.01.27,Мастер сельскохозяйственного производства,35.01.27 Мастер сельскохозяйственного производ...,9 кл.,25.0,1


In [38]:
base_df_groupby = union_df.groupby(['Наименование']).agg({'for_counting':sum})
base_df_groupby['for_counting'] = base_df_groupby['for_counting'].apply(int)
base_df_groupby = base_df_groupby.reset_index()
base_df_groupby.rename(columns={'for_counting':'Подано заявлений'},inplace=True)


In [39]:
base_df

,Код,Наименование,База,Количество мест,Подано заявлений
9,23.01.17,Мастер по ремонту и обслуживанию автомобилей,9 кл.,25,74
13,35.01.27,Мастер сельскохозяйственного производства,9 кл.,25,20
5,15.01.35,Мастер слесарных работ,9 кл.,25,32
0,23.01.09,Машинист локомотива,9 кл.,100,172
10,08.02.09,"Монтаж, наладка и эксплуатация электрооборудов...",9 кл.,25,33
1,43.01.06,Проводник на железнодорожном транспорте,9 кл.,25,76
4,15.01.05,Сварщик (ручной и частично механизированной св...,9 кл.,50,95
3,43.02.06,Сервис на транспорте (по видам транспорта),9 кл.,25,69
7,23.01.10,Слесарь по обслуживанию и ремонту подвижного с...,9 кл.,50,64
2,23.02.06,Техническая эксплуатация подвижного состава же...,9 кл.,25,27


In [36]:
base_df_groupby = union_df.groupby(['Наименование']).agg({'for_counting':sum})
base_df_groupby['for_counting'] = base_df_groupby['for_counting'].apply(int)
base_df_groupby = base_df_groupby.reset_index()
base_df_groupby.rename(columns={'for_counting':'Подано заявлений'},inplace=True)

base_df = base_df.merge(base_df_groupby,how='inner',left_on='Наименование',right_on='Наименование')
base_df.sort_values(by='Наименование',inplace=True)
base_df.rename(columns={'Наименование':'Наименование образовательной программы'})
base_df.drop(columns='Направление подготовки',inplace=True)
